In [14]:
%cd /content/drive/MyDrive/Colab Notebooks/regaetton_songs_nlp

/content/drive/MyDrive/Colab Notebooks/regaetton_songs_nlp


# Download the spanish embeddings


In [8]:
# %cd embeddings
# Embeddings
# !wget -q http://dcc.uchile.cl/~jperez/word-embeddings/fasttext-sbwc.100k.vec.gz

# unzip
# !gzip -d -q fasttext-sbwc.100k.vec.gz


In [3]:
normalized_eval_path = '/content/drive/MyDrive/Colab Notebooks/regaetton_songs_nlp/data/normalized_eval_lyrics.csv'
split_train_path = '/content/drive/MyDrive/Colab Notebooks/regaetton_songs_nlp/data/normalized_train_split.csv'
scores_path = '/content/drive/MyDrive/Colab Notebooks/regaetton_songs_nlp/data/scores.csv'

In [10]:
import nltk
import pandas as pd

from nltk import word_tokenize
from gensim.models import KeyedVectors
from gensim.models.wrappers import FastText
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, recall_score

nltk.download('punkt')

def add_score(df, model, parameters, accuracy, recall):
    row = {'model': model, 'parameters': parameters,
           'accuracy': accuracy, 'recall': recall}
    df = df.append(row, ignore_index=True)

    return df

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Load the data and the embeddings

In [27]:
train = pd.read_csv(split_train_path)
eval = pd.read_csv(normalized_eval_path)

In [21]:
spanish_emb_path = '/content/drive/MyDrive/Colab Notebooks/regaetton_songs_nlp/embeddings/fasttext-sbwc.100k.vec'
spanish_emb = KeyedVectors.load_word2vec_format(spanish_emb_path)

In [26]:
emb_vocab = spanish_emb.vocab.keys()

# The first thing that we want to check is the amount of overlap between our vocab and the embeddings vocab.


In [ ]:
lyrics_vocab = list()
def get_vocab(lyrics):
    global lyrics_vocab

    words = lyrics.split()
    for word in words:
        if word not in lyrics_vocab:
            lyrics_vocab.append(word)

train.lyrics.apply(get_vocab)


In [30]:
def intersection(lst1, lst2): 
    overlap_words = set(lst1).intersection(lst2) 
    return overlap_words
overlap = intersection(emb_vocab, lyrics_vocab)
print(f'The amount of words in our vocab is {len(lyrics_vocab)}\nThe amount of words in both is {len(overlap)}') 

The amount of words in our vocab is 44291
The amount of words in both is 23350


In [32]:
print(list(overlap)[:100])

['lecciones', 'vendaval', 'manes', 'summer', 'carta', 'hechizos', 'saludo', 'lotería', 'sésamo', 'chiquillo', 'acontecimiento', 'trueno', 'cargar', 'pegaso', 'paz', 'focos', 'viñas', 'moleste', 'alzó', 'permito', 'mesita', 'progresado', 'tarima', 'requiere', 'sinceridad', 'enterarme', 'buzo', 'desenfreno', 'evoca', 'dijo', 'valdez', 'livia', 'sudo', 'creerlo', 'ena', 'evangelio', 'sinceramente', 'mitad', 'arrepiento', 'rumora', 'porción', 'desperate', 'reclame', 'lucho', 'hablemos', 'osamenta', 'podrida', 'determinación', 'queremos', 'suspirar', 'educado', 'recibes', 'galo', 'apartamentos', 'saqueando', 'motoristas', 'guardián', 'jaque', 'iluminaba', 'dancer', 'lados', 'mp', 'blade', 'cedió', 'unida', 'royal', 'bitch', 'quedaban', 'engañado', 'niños', 'maker', 'patiño', 'activos', 'ceder', 'comandando', 'advertimos', 'hermanos', 'anticipación', 'cae', 'acerqué', 'seeger', 'vel', 'bailar', 'sevilla', 'obscuro', 'imperfecto', 'oficios', 'constelación', 'continuan', 'talentoso', 'puesto',

The overlap is not enough to get a good embedding representation of each song, so the most recommendable thing to do is to create specific embeddings for our task.

Some of the reasons for this poor overlap can be :
- obviously that the domain in which the embeddings were trained is very different to our task domain. 
- Because of the nature of our data, which is input by users, we have a lot of spelling mistakes that, although, we have tried to solve some still pass the different checks. (example above: ena, qu, vel)
- finally, some lyrics have content in other languages, we also have done a lot of preprocessing to delete other languages but spanish. However, there are still some words that manage to past the different checks. (example above: summer) 